# TINT Demonstration


In [30]:
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import tempfile
import os
import shutil

import pyart
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate

## Data


In [2]:
# Let's get some nexrad data from Amazon S3
keys = get_nexrad_keys('khgx', start='20150704_130000', end='20150704_140000')

Found 14 files.


In [3]:
fig = plt.figure(figsize=(10, 7))


def plot_ppi(key):
    plt.clf()
    radar = read_nexrad_key(key)
    display = pyart.graph.RadarMapDisplay(radar)
    display.plot_ppi_map('reflectivity', resolution='l',
                         sweep=3, fig=fig, width=200000, height=200000)
    del display, radar


ppi_anim = FuncAnimation(fig, plot_ppi, frames=keys, interval=1000)
HTML(ppi_anim.to_jshtml())

### Disclaimer
If you don't want this data to remain on your filesystem, make sure you run the last cell in this notebook. If your kernel crashes for some reason, you can manually remove the the path printed after the following cell.

In [10]:
# These radar volumes look interesting.
# Let's make a function that maps them to a cartesian grid using pyart.


def get_grid(radar):
    """ Returns grid object from radar object. """
    grid = pyart.map.grid_from_radars(
        radar, grid_shape=(17, 200, 200),
        grid_limits=((0, 17000), (-100000, 100000), (-100000, 100000)),
        fields=['reflectivity'], gridding_algo='map_gates_to_grid')
    return grid


# Let's write these grids to a temporary location so that we can reuse them.
tmp_dir = tempfile.mkdtemp()
print('temporary directory:', tmp_dir)
filenames = []
for num, key in enumerate(keys):
    print('saving grid', num)
    radar = read_nexrad_key(key)
    grid = get_grid(radar)
    name = os.path.join(tmp_dir, 'grid_' + str(num).zfill(3) + '.nc')
    filenames.append(name)
    pyart.io.write_grid(name, grid)
    del radar, grid

temporary directory: /tmp/tmp3v93dupk
saving grid 0
saving grid 1
saving grid 2
saving grid 3
saving grid 4
saving grid 5
saving grid 6
saving grid 7
saving grid 8
saving grid 9
saving grid 10
saving grid 11
saving grid 12
saving grid 13


In [15]:
grids = (pyart.io.read_grid(fn) for fn in filenames)

## Let's do some tracking

In [17]:
tracks_obj = Cell_tracks()
tracks_obj.params

{'FIELD_THRESH': 32,
 'FLOW_MARGIN': 10000,
 'GS_ALT': 1500,
 'ISO_SMOOTH': 3,
 'ISO_THRESH': 8,
 'MAX_DISPARITY': 999,
 'MAX_FLOW_MAG': 50,
 'MAX_SHIFT_DISP': 15,
 'MIN_SIZE': 8,
 'SEARCH_MARGIN': 4000}

In [18]:
tracks_obj.get_tracks(grids)

Writing tracks for scan 0
Writing tracks for scan 1
Writing tracks for scan 2
Writing tracks for scan 3
Writing tracks for scan 4
Writing tracks for scan 5
Writing tracks for scan 6
Writing tracks for scan 7
Writing tracks for scan 8
Writing tracks for scan 9
Writing tracks for scan 10
No cells found in scan 11
No cells found in scan 12
Writing tracks for scan 13


time elapsed 0.0 minutes


In [28]:
tracks_obj.tracks

area   grid_x   grid_y isolated      lat      lon        max  \
scan uid                                                                      
0    0    20.201510  172.250   10.750    False  28.6699 -94.3319  40.597511   
     1    18.181359  132.667   49.722     True  29.0241 -94.7325  39.694050   
     2    12.120906  151.417   95.167    False  29.4302 -94.5443  40.934441   
     3     8.080604  169.875  164.875    False  30.0619 -94.3425  37.494282   
1    0    27.272039  172.556   12.407     True  28.6789 -94.3215  43.382648   
     1    16.161208  133.875   51.125     True  29.0331 -94.7221  43.173172   
     2    12.120906  153.333   97.667     True  29.4572 -94.5234  41.452248   
2    0    15.151133  173.400   13.400     True  28.6879 -94.3215  39.828522   
     1    16.161208  134.812   52.688     True  29.0511 -94.7117  42.427731   
     2    14.141057  154.857   99.357    False  29.4661 -94.5026  40.721413   
3    4    12.120906   90.500    0.667     True  28.5816 -95.1765  42.342644   
     1    20.201510  135.950   55.000     True  29.0692 -94.7013  42.534935   
     2    14.141057  156.143  101.429     True  29.4842 -94.4921  41.281303   
4    4    27.272039   89.926    3.148     True  28.5997 -95.1765  41.850006   
     1    14.141057  136.643   55.929     True  29.0782 -94.6909  39.633705   
     2    12.120906  157.500  103.500    False  29.5112 -94.4712  39.756844   
5    4    14.141057   89.643    1.286    False  28.5816 -95.1765  38.467335   
     5    15.151133   89.533    8.600    False  28.6539 -95.1766  38.503956   
     2    10.100755  158.800  105.300    False  29.5202 -94.4607  37.719643   
6    5    16.161208   89.625   10.938    False  28.6720 -95.1766  40.593254   
     2     8.080604  159.750  106.500    False  29.5292 -94.4503  38.064781   
7    5    22.221661   89.636   13.227    False  28.6900 -95.1766  41.553699   
     6     8.080604  158.125  112.125    False  29.5835 -94.4707  42.300728   
8    5    22.221661   89.818   15.591    False  28.7172 -95.1766  41.399055   
     6     9.090680  159.000  114.000    False  29.6015 -94.4602  43.172081   
9    5    16.161208   90.125   17.750    False  28.7352 -95.1767  39.911072   
10   5    13.130982   90.154   20.000    False  28.7533 -95.1767  40.023804   
13   7     8.080604  131.500   55.000     True  29.0693 -94.7427  38.724812   
     8    10.100755  168.800  122.900     True  29.6823 -94.3557  40.346382   
     9    11.110831  172.182  141.273    False  29.8449 -94.3233  41.050888   

          max_alt                time         vol  
scan uid                                           
0    0     5.3125 2015-07-04 13:03:40  110.540138  
     1     4.2500 2015-07-04 13:03:40   50.440645  
     2     4.2500 2015-07-04 13:03:40   33.269362  
     3     4.2500 2015-07-04 13:03:40   34.342567  
1    0     5.3125 2015-07-04 13:07:58  128.784627  
     1     5.3125 2015-07-04 13:07:58   67.611929  
     2     4.2500 2015-07-04 13:07:58   39.708593  
2    0     4.2500 2015-07-04 13:12:16   59.026287  
     1     8.5000 2015-07-04 13:12:16   65.465519  
     2     4.2500 2015-07-04 13:12:16   50.440645  
3    4     4.2500 2015-07-04 13:16:22   54.733466  
     1     5.3125 2015-07-04 13:16:22   60.099492  
     2     5.3125 2015-07-04 13:16:22   50.440645  
4    4     5.3125 2015-07-04 13:20:21  119.125780  
     1     4.2500 2015-07-04 13:20:21   38.635388  
     2     8.5000 2015-07-04 13:20:21   39.708593  
5    4     4.2500 2015-07-04 13:24:33   54.733466  
     5     4.2500 2015-07-04 13:24:33   60.099492  
     2     3.1875 2015-07-04 13:24:33   32.196157  
6    5     5.3125 2015-07-04 13:28:45   71.904750  
     2     2.1250 2015-07-04 13:28:45   25.756925  
7    5     4.2500 2015-07-04 13:32:44   92.295649  
     6     4.2500 2015-07-04 13:32:44   30.049746  
8    5     4.2500 2015-07-04 13:36:44   90.149239  
     6     4.2500 2015-07-04 13:36:44   35.415772  
9    5     4.2500 2015-07-04 13:40:43   61.172698  
10   5     3.1875 2015-07-04 1

In [21]:
# lets make another generator for the animator
grids = (pyart.io.read_grid(fn) for fn in filenames)


In [26]:
os.chdir('/home/mhpicel/Practice')

In [27]:
animate(tracks_obj, grids, 'tint_demo_anim')

Animating 14 frames
Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13


## Clean Up

In [31]:
shutil.rmtree(tmp_dir)